In [ ]:
from typing import List, Dict, Any, Callable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import Solution, Problem, Minimize, Maximize, ObjectiveFunction
from heuristics import HillClimbing

ModuleNotFoundError: No module named 'tqdm'

# 1. Solução

As soluções do problema de alocação de empresas à projetos estão modeladas na classe `Solution`, que é construída a partir de dois parâmetros:

- `n_companies`: o número de empresas a ser modelado;
- `n_projects`: o número de projetos a ser modelado.

De posse dessas duas informações, é criado então um atributo `x`, um array do Numpy que armazena os valores da solução codificada. Essa codificação transforma a matriz esparsa $m \times n$ em um array 1-D cujos índices e valores indicam a linha e coluna de posição de uma alocação, respectivamente. Dessa forma, a restrição de ter apenas um projeto alocado a somente uma empresa é obedecida sempre.

De forma a facilitar a visualização da solução baseada em alocações empresa $\times$ projeto, o método `sparse()` retorna um array 2-D com a matriz original da codificação.

Por fim, a função de vizinhança é implementada no método `move()`, que escolhe aleatoriamente dois índices da solução e realiza a troca entre os valores, resultando assim em uma solução vizinha a anterior.

A execução a seguir demonstra a criação de uma solução que considera 9 empresas e 9 projetos, mostrando suas versões densa e esparsa, assim como é realizada a movimentação para uma solução vizinha.


In [2]:
solution = Solution(n_companies=9, n_projects=9)

print("Initial solution: ")
print(solution)
print(solution.sparse())

solution.move()

print("\nNeighbor solution: ")
print(solution)
print(solution.sparse())

Initial solution: 
[6 2 1 5 8 4 3 9 7]
[[0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0]]

Neighbor solution: 
[9 2 1 5 8 4 3 6 7]
[[0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]]


# 2. Função objetivo

Antes de definir a função objetivo, é importante armazenar a matriz de custos `cost_matrix`, que armazena os custos que cada empresa teria ao executar cada projeto, considerando a disposição dos índices como referência em relação às soluções geradas.

In [3]:
cost_matrix = np.array([
    [12, 18, 15, 22, 9, 14, 20, 11, 17],
    [19, 8, 13, 25, 16, 10, 7, 21, 24],
    [6, 14, 27, 10, 12, 19, 23, 16, 8],
    [17, 11, 20, 9, 18, 13, 25, 14, 22],
    [10, 23, 16, 14, 7, 21, 12, 19, 15],
    [13, 25, 9, 17, 11, 8, 16, 22, 20],
    [21, 16, 24, 12, 20, 15, 9, 18, 10],
    [8, 19, 11, 16, 22, 17, 14, 10, 13],
    [15, 10, 18, 21, 13, 12, 22, 9, 16]
])

A função objetivo para o problema de alocação de empresas à projetos foi modelada na classe `ObjectiveFunction`, que recebe como parâmetro uma matriz de custos. O objeto instanciado pode então ser chamado, com a passagem de uma nova solução como parâmetro, calculando assim o custo da solução.

A execução a seguir demonstra o uso da função objetivo, calculando o custo para uma solução inicial e uma solução vizinha.

In [4]:
objective_function = ObjectiveFunction(cost_matrix)

initial_cost = objective_function(solution)
print(f"Initial cost: {initial_cost}")

solution.move()

new_cost = objective_function(solution)
print(f"New cost: {new_cost}")

Initial cost: 148
New cost: 148


# 3. Problema

As considerações de um problema de otimização foram modeladas em uma superclasse `Problem`, um esboço para a implementação das classes que de fato definem o problema, `Minimize` e `Maximize`. Ao instanciar um desses tipos de problema, o objeto fica disponível para computar a condição de melhoria de custos de qualquer algoritmo de otimização ao retornar um booleano que responda ao problema, bastando passar os seguintes parâmetros:

- `new_cost`: novo custo alcançado por uma solução vizinha à atual;
- `best_cost`: melhor custo encontrado pelo algoritmo até o momento.

A execução a seguir demonstra o uso da classe para um problema de minimização.

In [5]:
problem = Minimize()
problem(new_cost=71, best_cost=72)

True

# 4. Heurísticas

## 4.1. Hill Climbing

In [ ]:
hc = HillClimbing(
    problem=Minimize(),
    objective_function=objective_function,
    cost_matrix=cost_matrix,
    max_iterations=10_000,
    patience=100
)

hc.run()

hc

Iteration: 10000

Best cost: 87

Best solution: [8 2 6 9 3 4 7 5 1]

Allocation:
[[0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]